In [ ]:
project_path = "/media/vijinim/data/Experiments/Data/FUDAN/Uniprot-12/"

In [ ]:
import re
import xlsxwriter
import collections

from Bio import SeqIO

In [ ]:
contig_lengths = {}

for index, record in enumerate(SeqIO.parse(project_path+"all_genes.fna", "fasta")):
    gene_id = record.id.strip().split("|")[0]
    contig_lengths[gene_id] = len(record.seq)

# Get kraken result

In [ ]:
species_len = {}

unclassified = []

taxid_contigs = {}

annotated = 0

with open(project_path+"MEGAHIT_new_genes_kraken_res_0.1.txt", "r") as myfile:
    for line in myfile.readlines():
        strings = line.strip().split("\t")
        
        gene_id = strings[1].strip().split("|")[0]
        
        if strings[0] == "C":
            
            species_strings = strings[2].split(" ")
            
            if len(species_strings) > 3 and "cellular organisms" not in strings[2]:
                
                annotated += 1
                
                taxid = species_strings[-1][:-1]
                
                if taxid not in taxid_contigs:
                    taxid_contigs[taxid] = [gene_id]
                else:
                    taxid_contigs[taxid].append(gene_id)
            
            if strings[2] not in species_len:
                species_len[strings[2]] = contig_lengths[gene_id]
            else:
                species_len[strings[2]] += contig_lengths[gene_id]
                
        else:
            unclassified.append(gene_id)

In [ ]:
print("Total number of genes:", len(contig_lengths))
print("Number of annotated genes:", annotated)
print("Number of taxa:", len(species_len))

In [ ]:
sorted_species = {k: v for k, v in sorted(species_len.items(), reverse=True, key=lambda item: item[1])}

species_list = []
values_list = []
taxid_list = []

taxid_species = {}
taxid_total_len = {}

annotated_genes = []

for key, value in sorted_species.items():
    key_strings = key.split(" ")
    
    if len(key_strings) > 3 and "cellular organisms" not in key:
        
        taxid = key_strings[-1][:-1]
        
        species_name = key.split(" (taxid ")[0]
        
        if taxid not in taxid_list:
            taxid_list.append(taxid)
            
        if taxid not in taxid_species:
            taxid_species[taxid] = species_name
            
        if taxid not in taxid_total_len:
            taxid_total_len[taxid] = value
            
        annotated_genes += taxid_contigs[taxid]
        
        species_list.append(species_name)
        values_list.append(value)
        print(species_name, value)

print(len(species_list))

In [ ]:
print("Total number of genes:", len(contig_lengths))
print("Number of annotated genes upto species level:", len(annotated_genes))
print(len(annotated_genes)/len(contig_lengths))

In [ ]:
annotated = 0

for taxid in taxid_list:
    annotated += len(taxid_contigs[taxid])
print(annotated)

In [ ]:
gene_species = {"GeneID":"Annotation"}

for taxid in taxid_list:
    
    for gene_id in taxid_contigs[taxid]:

        gene_species[gene_id] = "["+taxid_species[taxid]+"]"
            
            
for gene_id in contig_lengths:
    if gene_id not in annotated_genes:
        gene_species[gene_id] = "-"

# Save result

In [ ]:
od = collections.OrderedDict(sorted(gene_species.items()))

# Create a workbook and add a worksheet.
workbook = xlsxwriter.Workbook(project_path+"MEGAHIT_new/genes.species.kraken.xlsx")
worksheet = workbook.add_worksheet()

# Start from the first cell. Rows and columns are zero indexed.
row = 0
col = 0

print(len(od.items()))

# Iterate over the data and write it out row by row.
for gene, species in od.items():
    worksheet.write(row, col,     gene)
    worksheet.write(row, col + 1, species)
    row += 1
    
workbook.close()